Лабораторная работа №6
Жидков А. В.
ДПИ22-1

UDP-Сервер

In [ ]:
import socket
import threading

class UDPChatServer:
    def __init__(self, host='0.0.0.0', port=12345):
        self.host = host
        self.port = port
        self.server_socket = None
        self.clients = set()

    def start(self):
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.server_socket.bind((self.host, self.port))
        print("Сервер запущен. Ожидание сообщений...")

        try:
            while True:
                data, addr = self.server_socket.recvfrom(1024)
                message = data.decode()
                print(f"Получено сообщение от {addr}: {message}")

                if addr not in self.clients:
                    self.clients.add(addr)

                # Рассылка сообщения всем клиентам
                self.broadcast(message, addr)
        except KeyboardInterrupt:
            print("Остановка сервера.")
        finally:
            self.server_socket.close()

    def broadcast(self, message, sender_addr):
        for client in self.clients:
            if client != sender_addr:
                self.server_socket.sendto(message.encode(), client)

if __name__ == "__main__":
    server = UDPChatServer()
    server.start()


Сервер запущен. Ожидание сообщений...


UDP-Клиент

In [ ]:
import socket
import threading

class UDPChatClient:
    def __init__(self, host='127.0.0.1', port=12345):
        self.host = host
        self.port = port
        self.client_socket = None

    def start(self):
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

        # Запускаем поток для приема сообщений
        threading.Thread(target=self.receive_messages, daemon=True).start()

        print("Соединение с сервером установлено.")
        while True:
            message = input("Введите сообщение (или 'exit' для выхода): ")
            if message.lower() == 'exit':
                break
            self.client_socket.sendto(message.encode(), (self.host, self.port))

        self.client_socket.close()
        print("Разрыв соединения с сервером.")

    def receive_messages(self):
        while True:
            try:
                data, _ = self.client_socket.recvfrom(1024)
                print(f"\nНовое сообщение: {data.decode()}")
            except OSError:
                break

if __name__ == "__main__":
    client = UDPChatClient()
    client.start()
